**Exemplo Para Leitura Via Socket e Gravação em Arquivo**

In [58]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window
from pyspark.sql.functions import col


spark = SparkSession \
    .builder \
    .appName("StructuredWorkWordCountToFile") \
    .getOrCreate()

In [59]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .option('includeTimestamp', 'true')\
    .load()

# Divide as linhas recebidas em cada palavra
words = lines.select(
        explode(split(lines.value, ' ')).alias('word'),
        col("timestamp")#lines.timestamp
    )

# cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
windowedCounts = words \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(col("timestamp"), "10 minutes", "5 minutes"),
        words.word) \
    .count()

In [60]:
#confere o tipo de dataframe
windowedCounts.isStreaming

True

In [63]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = windowedCounts\
    .writeStream\
    .outputMode('append')\
    .format('json')\
    .option('truncate', 'false')\
    .option("path", "/home/tulio/Documents/IGTI/PFC/drive-download/output_csv")\
    .option("checkpointLocation", "/home/tulio/Documents/IGTI/PFC/drive-download/output_csv/chekPoint")\
    .start()



query.awaitTermination() #aguarda até que a "streaming query" termine 

KeyboardInterrupt: 

Para rodar é necessário ter o "nc" instalado e utiliar o comando antes de iniciar a célula anterior

nc -lk 9999